'''
작업 제목: 정적인 페이지에서 데이터 스크래핑 BeautifulSoup 이용
작업 목표: imdb에서 워킹데드 드라마 설명 페이지에서 시즌 별로 드라마 제목, 방영일, 리뷰수, 평점 데이터 가져오기 (https://www.imdb.com/title/tt1520211/episodes?season=1/)
연습 출처: https://www.snugarchive.com/blog/python-web-scraping/
'''

In [1]:
import re
import csv
import requests
from bs4 import BeautifulSoup

In [ ]:
# 데이터를 받아와서 담을 그릇(csv 파일 준비)

In [2]:
filename="theWalkingDead_season_info.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

In [6]:
# 뭘 가져올지 생각하고 원하는 데이터 셋의 구조(열별로 뭘 담을지) 결정
# 결정 후에 만들어 놓은 그릇에 제목 행까지는 데이터 써 놓기 witer.writerow()

In [3]:
attributes = ['시즌','제목', '방영일', '리뷰수','평점']
writer.writerow(attributes)

18

In [ ]:
# 정보들이 시즌별로 다른 페이지에 흩어져 있어, 총 11 시즌 까지 있으므로 11번 반복해서 돌아다니면서 11개의 페이지의 html을 가져와야함


In [4]:
for season in range(1,12):
    url = f"https://www.imdb.com/title/tt1520211/episodes?season={season}" # 짠~
    res = requests.get(url)
    res.raise_for_status() # res.status_code==200 이어야 진행, 진행된다면 문제 없는거
    soup = BeautifulSoup(res.text,"lxml")

    episodes = soup.find_all("div", attrs={"itemprop":"episodes"})

    for episode in episodes:
        title = episode.find("a",attrs={"itemprop":"name"}).get_text()
        airdate = episode.find("div", attrs={"class":"airdate"}).get_text()
        total_votes = episode.find("span", attrs={"class":"ipl-rating-star__total-votes"}).get_text()
        rating = episode.find("span", attrs={"class":"ipl-rating-star__rating"}).get_text()
        # 긁어온 숫자들 예쁘게 다듬기
        airdate = airdate.strip()
        # strip() = lstrip() + rstrip() 앞뒤 공백 없애주는. 괄호안에 특정문자 넣으면 맨앞이나 맨뒤에서 그 문자를 없앰
        # airdate = re.sub(r"^\s+|\s+$","",airdate) # 효과가 같음 
        total_votes = re.sub("[()]","", total_votes) # 앞뒤로 ()괄호 삭제
        data_row = [season, title, airdate, total_votes, rating]
        print(data_row)
        writer.writerow(data_row)

[1, 'Days Gone Bye', '8 Oct. 2019', '27,510', '9.2']
[1, 'Guts', '7 Nov. 2010', '17,310', '8.6']
[1, 'Tell It to the Frogs', '14 Nov. 2010', '15,841', '8.2']
[1, 'Vatos', '21 Nov. 2010', '15,474', '8.5']
[1, 'Wildfire', '28 Nov. 2010', '14,884', '8.1']
[1, 'TS-19', '5 Dec. 2010', '16,211', '8.6']
[2, 'What Lies Ahead', '16 Oct. 2011', '14,539', '8.5']
[2, 'Bloodletting', '23 Oct. 2011', '13,213', '7.9']
[2, 'Save the Last One', '30 Oct. 2011', '13,182', '8.3']
[2, 'Cherokee Rose', '6 Nov. 2011', '12,793', '7.5']
[2, 'Chupacabra', '13 Nov. 2011', '12,894', '8.0']
[2, 'Secrets', '20 Nov. 2011', '12,533', '7.6']
[2, 'Pretty Much Dead Already', '27 Nov. 2011', '18,230', '9.2']
[2, 'Nebraska', '12 Feb. 2012', '12,704', '8.2']
[2, 'Triggerfinger', '19 Feb. 2012', '12,280', '8.2']
[2, '18 Miles Out', '26 Feb. 2012', '12,719', '8.4']
[2, 'Judge, Jury, Executioner', '4 Mar. 2012', '12,950', '8.5']
[2, 'Better Angels', '11 Mar. 2012', '15,500', '9.1']
[2, 'Beside the Dying Fire', '18 Mar. 2012',

In [5]:
f.close()
# 닫아야 데이터가 들어가